In [1]:
#!git clone https://github.com/chunfengc/ASNet.git

In [1]:
!pwd

/home/awas/NLA/NLA_Proj/ASNet


In [2]:
import os
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn

import time
import sys

#from ASNet.active_subspace import *
from datasets.dataloader_cifar import *
from ASNet.utils import *
from ASNet.PCEModel import *
from ASNet.ASModel import *
from ASNet.FineTuning import *
from ASNet.ASNet import *
from model.resnet import *# resnet20,resnet32,resnet44,resnet56,resnet110#1202
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
torch.cuda.set_device(0)
CUDA_VISIBLE_DEVICES=0

In [3]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device = torch.device('cpu')


In [4]:
!nvidia-smi

Tue Dec 15 22:21:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650    Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8     3W /  N/A |    352MiB /  3914MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!kill -9 22720

/bin/bash: строка 0: kill: (22720) - Нет такого процесса


In [6]:
device

device(type='cuda', index=0)

In [7]:
# Load the dataset
bs = 200
train_loader = trainloader_resnet(bs)
test_loader = testloader_resnet(bs)
test_labels = torch.tensor(test_loader.dataset.targets)
train_labels = torch.tensor(train_loader.dataset.targets)

Files already downloaded and verified


In [8]:
modelname = 'resnet110'
if modelname == 'resnet110':
    model = resnet110()
elif modelname == 'resnet56':    
    model = resnet56()
elif modelname == 'resnet44':
    model = resnet44()
elif modelname == 'resnet20':
    model = resnet20()

In [9]:
model

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [10]:
model = torch.nn.DataParallel(model)

In [11]:
# !wget https://github.com/akamaster/pytorch_resnet_cifar10/raw/master/pretrained_models/resnet110-1d1ed7c2.th

In [12]:
ls

ASNet/       experiment/  resnet110-1d1ed7c2.th
ASNet.ipynb  model/       resnet110_CIFAR_10_ASModel_nAS_50_cutID_31.pth
datasets/    README.md


In [13]:
# pretrained = 'resnet110-1d1ed7c2.th'.format(modelname)
pretrained = 'resnet110-1d1ed7c2.th'

if torch.cuda.is_available():
    tmp = torch.load(pretrained)
else:
    tmp = torch.load(pretrained, torch.device('cpu'))
model.load_state_dict(tmp['state_dict'])
#model.load_state_dict(tmp['state_dict'], strict=False)
model.to(device)
model.eval()
seq_model = get_seq_model_resnet(model)
seq_model.to(device)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): BasicBlock(
    (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )
  (2): BasicBlock(
    (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, t

In [14]:
train_max_batch = len(train_loader)

######### Training
nAS = 50
r_max = max(200,nAS)
loss = F.cross_entropy
add_softmax = False
p = 2

In [15]:
device

device(type='cuda', index=0)

In [16]:
cut_idx =31
cut_idx_seq = cut_idx
pre_model = seq_model[:cut_idx_seq].eval().to(device)
post_model = seq_model[cut_idx_seq:].eval().to(device)


# Load or compute the AS Layer
fAS_name = '%s_CIFAR_10_ASModel_nAS_%d_cutID_%d.pth'%(modelname,nAS,cut_idx)
if os.path.isfile(fAS_name):
    AS_models, Sigma = torch.load(fAS_name)
    print('    AS layer loaded')
    
else:
    AS_models, Sigma = get_ASModel_FD(seq_model, train_loader,
              cut_idx_seq, train_max_batch,r_max,device)
    torch.save([AS_models,Sigma], fAS_name) 
    print('    AS layer computed')

for key in AS_models.keys():
    AS_model = AS_models[key]
AS_model.change_r(nAS)
#print(AS_model.V.device)

Z_train, y_train = compute_Z_AS_space(AS_model, pre_model, post_model, train_loader, 
        train_max_batch,device=device)


print('Z_train, Z_test generated')
mean = torch.mean(Z_train,0).to(device)
var =  torch.std(Z_train,0).to(device)
PCE_model = PCEModel(mean,var,nAS,p, device=device)

Z_train = Z_train.to(device)
y_train = y_train.to(device)
train_labels = train_labels.to(device)
coeff, training_score_LR, training_score_labels = PCE_model.Training(Z_train, y_train,train_labels[:Z_train.shape[0]])
PCE_coeff = torch.tensor(coeff,dtype=torch.float32).to(device)
print('Training score =', training_score_LR, \
      'label score', training_score_labels)


basis_layer = BasisLayer(PCE_model,device=device)
ASNet = ASNET(pre_model, AS_model, basis_layer,PCE_coeff).to(device)

print('Training and testing error for ASNet without re-training')

    AS layer loaded
Z_train, Z_test generated
Training score = 0.6888949826711134 label score 0.10088
Training and testing error for ASNet without re-training


https://github.com/akamaster/pytorch_resnet_cifar10

In [17]:
print(next(ASNet.parameters()).device)

cuda:0


In [18]:
#from tqdm import tqdm.notebook
optimizer = torch.optim.Adam([{'params': ASNet.premodel.parameters(),'lr': 1e-3},
                            {'params': ASNet.AS.parameters(), 'lr': 1e-5},
                            {'params': ASNet.PCE.parameters(), 'lr': 1e-5}])

epochs = 50   
filename = 'CIFAR10_ResNet110_ASNet'+\
        '_epoch_%d_nAS_%d_cutID_%d.pth'%(epochs,nAS,cut_idx)

if os.path.isfile(filename): 
    [ASNet_pretrained,train_loss,test_loss] = torch.load(filename)
    ASNet.load_state_dict(ASNet_pretrained)
    print('ASNet trained {} epoches is loaded'.format(epochs))
else:
    train_loss = []
    test_loss = []
#    compute_loss(ASNet, device, train_loader)
    train_loss.append(compute_loss(ASNet, device, train_loader))
    test_loss.append(compute_loss(ASNet, device, test_loader))
    
    for epoch in range(1, epochs + 1):
        print('EPOCH {}'.format(epoch))
        print(device)
        train_loss.append(train_kd(ASNet, model, device, train_loader, optimizer, train_max_batch, 
                  alpha=0.2, temperature=1., epoch=epoch))
        test_loss.append(compute_loss(ASNet, device, test_loader))

    torch.save([ASNet.state_dict(),train_loss,test_loss],filename)

 Top 1:  Accuracy: 39314.0/50000 (78.63%)
 Top 1:  Accuracy: 7880.0/10000 (78.80%)
EPOCH 1
cuda:0


RuntimeError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 0; 3.82 GiB total capacity; 2.55 GiB already allocated; 8.75 MiB free; 2.63 GiB reserved in total by PyTorch)

In [ ]:
epochs_sparse = 100 
print_every = 10
lmd = 0.1
lmd2 = 0.1
optimizer = torch.optim.SGD([{'params': ASNet.premodel.parameters(), 'lr': 1e-4},
                        {'params': ASNet.AS.parameters(), 'lr': 1e-5},
                        {'params': ASNet.PCE.parameters(), 'lr': 1e-5}])
    
ASNet_storage = torch.zeros([epochs_sparse+1,3])
ASNet_flops = torch.zeros([epochs_sparse+1,3])

ASNet_storage[0,0],ASNet_storage[0,1],ASNet_storage[0,2] = [Total_param(ASNet.premodel),
                                    Total_param(ASNet.AS), Total_param(ASNet.PCE)]

ASNet_flops[0,0],ASNet_flops[0,1],ASNet_flops[0,2] = [Total_flops(ASNet.premodel,device),
                                    Total_flops(ASNet.AS,device), Total_flops(ASNet.PCE,device)]
# print('   acc: {:.2f} Pre nnz = {:.2f}, AS nnz={:.2f}, PCE nnz={:.2f}'.format(
#         test_loss[-1],ASNet_storage[0,0],ASNet_storage[0,1],ASNet_storage[0,2]))

In [ ]:

for epoch in range(1, epochs_sparse + 1): 
    print('EPOCH: {}'.format(epochs+epoch))
    train_loss.append(train_l1(ASNet, device, train_loader, optimizer, 
                          train_max_batch,lmd,lmd2,lr_decrease=None,epoch=epoch))
    test_loss.append(compute_loss(ASNet, device, test_loader))
    
    ASNet_storage[epoch,0] = Total_param_sparse(ASNet.premodel)
    ASNet_storage[epoch,1] = Total_param_sparse(ASNet.AS)
    ASNet_storage[epoch,2] = Total_param_sparse(ASNet.PCE)

    ASNet_flops[epoch,0],ASNet_flops[epoch,1],ASNet_flops[epoch,2] = [
        Total_flops_sparse(ASNet.premodel,device),
        Total_flops_sparse(ASNet.AS,device), Total_flops_sparse(ASNet.PCE,device)]
    
    print('   Pre nnz = {:.2f}/{:.2f}, AS nnz={:.2f}/{:.2f}, PCE nnz={:.2f}/{:.2f}'.format(
        ASNet_storage[epoch,0],ASNet_storage[0,0],ASNet_storage[epoch,1],
        ASNet_storage[0,1],ASNet_storage[epoch,2],ASNet_storage[0,2])) 

    if epoch%print_every ==0 or epoch==epochs_sparse:
        
        filename = 'CIFAR10_ResNet110_ASNet_sparse'+\
                    '_epoch_%d_nAS_%d_cutID_%d.pth'%(epochs_sparse,nAS,cut_idx)
        torch.save([ASNet.state_dict(),train_loss,test_loss,ASNet_storage,ASNet_flops,lmd,lmd2],filename)

EPOCH: 51


RuntimeError: ignored